# Getting familiar with API's and xlsx: Creating an equal-eeight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund. This is a basic project, yet valuable for private investors who want their investments to follow the world economy. A big thanks to Nick McCullum for teaching this topic!

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks= pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])] #delisted stocks


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol='AAPL'
api = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api).json()

# print(data)
print(type(data)) # it is a dictionary
print(data.keys()) # find all available keys


<class 'dict'>
dict_keys(['avgTotalVolume', 'calculationPrice', 'change', 'changePercent', 'close', 'closeSource', 'closeTime', 'companyName', 'currency', 'delayedPrice', 'delayedPriceTime', 'extendedChange', 'extendedChangePercent', 'extendedPrice', 'extendedPriceTime', 'high', 'highSource', 'highTime', 'iexAskPrice', 'iexAskSize', 'iexBidPrice', 'iexBidSize', 'iexClose', 'iexCloseTime', 'iexLastUpdated', 'iexMarketPercent', 'iexOpen', 'iexOpenTime', 'iexRealtimePrice', 'iexRealtimeSize', 'iexVolume', 'lastTradeTime', 'latestPrice', 'latestSource', 'latestTime', 'latestUpdate', 'latestVolume', 'low', 'lowSource', 'lowTime', 'marketCap', 'oddLotDelayedPrice', 'oddLotDelayedPriceTime', 'open', 'openTime', 'openSource', 'peRatio', 'previousClose', 'previousVolume', 'primaryExchange', 'symbol', 'volume', 'week52High', 'week52Low', 'ytdChange', 'isUSMarketOpen'])


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price=data['latestPrice']
market_cap=data['marketCap']

Let's try to print the price and marketcap to see if the data is legit.

In [6]:
price 

168.5

In [7]:
market_cap

2675125451350

Our price and marketcap data is actually randomized since we are working with a free API. But you can compare it with the real price and notixe that it is still within the neighbourhood of the real price. 

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']

Now let's create a blank pd dataframe that have these columns specified. 

In [9]:
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


Now, let's append the above data points into this data frame

In [10]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
   index=my_columns #this tells the append method which columns to add data to
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,168.5,2675125451350,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [16]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api).json()
    final_dataframe=final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index=my_columns),
    ignore_index=True
    )

KeyboardInterrupt: 

In [ ]:
final_dataframe

This is another way of testing if the stocks from our data is still a S&P 500 company, since the loop takes a long time we also include a progress bar. 

In [17]:
from tqdm import tqdm # to view progress bar

def get_tickers():
    
    stocks= pd.read_csv('sp_500_stocks.csv')
    
    all_data = []  #data from loop stored in this list
    
    for i, symbol in enumerate(tqdm(stocks.Ticker.unique(), desc="Looping over stock symbols")): # loop over unique tickers in data frame series
        # print(symbol)
        api = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
        
        try:
            data = requests.get(api).json()
            
        except:
            print(f"Not able to get data for {symbol} ")
        
        all_data.append(data) # A list of dictionarys
        
        df = pd.DataFrame(all_data)
        
        
    return df
    

In [18]:

stocks_info = get_tickers() # call function above
stocks_info

Looping over stock symbols:  27%|██▋       | 136/505 [01:03<02:49,  2.18it/s]

Not able to get data for DISCA 


Looping over stock symbols:  44%|████▎     | 220/505 [01:44<02:14,  2.12it/s]

Not able to get data for HFC 


Looping over stock symbols:  93%|█████████▎| 468/505 [03:44<00:17,  2.11it/s]

Not able to get data for VIAC 


Looping over stock symbols:  96%|█████████▌| 485/505 [03:52<00:10,  1.99it/s]

Not able to get data for WLTW 


Looping over stock symbols: 100%|██████████| 505/505 [04:02<00:00,  2.08it/s]


,avgTotalVolume,calculationPrice,change,changePercent,close,closeSource,closeTime,companyName,currency,delayedPrice,...,peRatio,previousClose,previousVolume,primaryExchange,symbol,volume,week52High,week52Low,ytdChange,isUSMarketOpen
0,1383027,close,3.07,0.02295,140.50,fiflcoia,1.727418e+12,Agilent Technologies Inc.,USD,138.910,...,33.55,131.96,1901950,TANR.E KCWCC YSOEKNXE GIHNO,A,2268204,184.66,116.36,-0.136697,False
1,37213233,close,-0.05,-0.00366,13.84,fciioafl,1.715133e+12,American Airlines Group Inc,USD,13.751,...,-4.88,14.09,35907789,DNQSAA,AAL,27515366,22.71,12.48,-0.245135,False
2,694549,close,2.69,0.01423,194.75,ioifcfal,1.730464e+12,Advance Auto Parts Inc,USD,193.780,...,21.61,197.02,782754,H TANWYXG E OO. NSNCKRECICEK,AAP,522588,241.71,165.00,-0.168775,False
3,72736104,close,5.18,0.03281,165.31,oiflcfia,1.678085e+12,Apple Inc,USD,164.100,...,27.57,162.96,82618959,DQSNAA,AAPL,103013902,183.28,133.70,-0.051694,False
4,5960331,close,-6.46,-0.04313,145.00,ciiafofl,1.661133e+12,Abbvie Inc,USD,145.050,...,20.76,150.28,6770441,K EYH .RGAOC CE WNIONCKNSXTE,ABBV,18038100,181.06,103.78,0.048760,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,1429512,close,-0.17,-0.00139,123.58,iofialfc,1.697653e+12,Yum Brands Inc.,USD,127.120,...,22.27,123.75,2067780,SOG CCNXNEOWRCAHNEYKK EIT.,YUM,1764256,138.92,112.42,-0.115031,False
501,1222535,close,-0.39,-0.00369,111.93,ifoaiclf,1.711431e+12,Zimmer Biomet Holdings Inc,USD,115.030,...,114.40,112.94,2302189,KNXT IAYE.O ECNROCSNHEKGWC,ZBH,1594529,171.50,101.90,-0.133066,False
502,365391,close,13.66,0.03901,374.70,fiafcoil,1.684329e+12,Zebra Technologies Corp. - Class A,USD,374.380,...,23.57,348.00,319416,ASADNQ,ZBRA,474727,644.00,289.80,-0.361759,False
503,1397451,close,1.08,0.02022,55.72,fafcoiil,1.722863e+12,Zions Bancorporation N.A,USD,55.720,...,5.91,56.05,1493671,DASANQ,ZION,1740350,75.58,48.21,-0.118438,False


In [19]:
stocks_info.columns

Index(['avgTotalVolume', 'calculationPrice', 'change', 'changePercent',
       'close', 'closeSource', 'closeTime', 'companyName', 'currency',
       'delayedPrice', 'delayedPriceTime', 'extendedChange',
       'extendedChangePercent', 'extendedPrice', 'extendedPriceTime', 'high',
       'highSource', 'highTime', 'iexAskPrice', 'iexAskSize', 'iexBidPrice',
       'iexBidSize', 'iexClose', 'iexCloseTime', 'iexLastUpdated',
       'iexMarketPercent', 'iexOpen', 'iexOpenTime', 'iexRealtimePrice',
       'iexRealtimeSize', 'iexVolume', 'lastTradeTime', 'latestPrice',
       'latestSource', 'latestTime', 'latestUpdate', 'latestVolume', 'low',
       'lowSource', 'lowTime', 'marketCap', 'oddLotDelayedPrice',
       'oddLotDelayedPriceTime', 'open', 'openTime', 'openSource', 'peRatio',
       'previousClose', 'previousVolume', 'primaryExchange', 'symbol',
       'volume', 'week52High', 'week52Low', 'ytdChange', 'isUSMarketOpen'],
      dtype='object')

Now let's visualize this (plotly is not typically installed by default in Anaconda). 

In [25]:
import plotly.express as px

def plot_top100_market_cap(df):

    df = df.copy()
    
    total_market_cap = df["marketCap"].sum() # in Trillions 
    
    print(f"Total market cap is: {total_market_cap /1000_000_000_000} Trillions")

    df["market_share"] = df["marketCap"] / total_market_cap
    
    # Sort and get top 100
    df = df.sort_values("marketCap", ascending=False).head(100)
    df2= df.sort_values("marketCap", ascending=False).head(10)
    
    fig = px.bar(df, x="symbol", y="market_share")
    fig.show()
    
    fig = px.pie(df2, names="symbol", values="market_share", title= "Market Share")
    fig.show()
    
    return df[["marketCap", "market_share"]]

plot_top100_market_cap(stocks_info)

Total market cap is: 35.650579131114 Trillions


,marketCap,market_share
3,2.756246e+12,0.077313
320,2.107640e+12,0.059119
36,1.435343e+12,0.040261
71,6.957074e+11,0.019515
177,5.694712e+11,0.015974
...,...,...
205,8.097339e+10,0.002271
65,8.070483e+10,0.002264
83,7.937766e+10,0.002227
313,7.935332e+10,0.002226


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of the code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [26]:
def chunks(lst,n): #split our pd series into "chunks" of size n
    """yield successive n-sized chunks from lst"""
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [27]:
symbol_groups=list(chunks(stocks['Ticker'],100)) #list of lists of size 100
#stocks['Ticker']

Now we will make a for loop that lists loops through every pd series within that list, execute a batch API call and for every stock in the list append the information for that stock to our final data frame.

In [28]:
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe=pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings[:4]:
    batch_api_call_url= f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
     #print(batch_api_call_url) #try pressing this link, should open up a browser with all the data
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','): #parse data for every stock in the symbol string
        final_dataframe=final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index=my_columns),
            ignore_index=True
        )
        
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.10,40533280753,N/A
1,AAL,14.00,9009657868,N/A
2,AAP,201.41,11814136920,N/A
3,AAPL,166.23,2689443502663,N/A
4,ABBV,150.22,262269902533,N/A
...,...,...,...,...
395,RL,101.64,7541509902,N/A
396,RMD,243.46,36301170259,N/A
397,ROK,264.21,29685602802,N/A
398,ROL,39.26,19041911938,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [29]:
portfolio_size=input('Enter the value of the portfolio:')
try:
    val=float(portfolio_size) 
    print(val)
except ValueError:
    print("That is not a number! \n Please try again:")
    portfolio_size=input('Enter the value of the portfolio:')
    val=float(portfolio_size)

Enter the value of the portfolio:
That is not a number! 
 Please try again:
Enter the value of the portfolio:10000000


In [30]:
position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/ final_dataframe.loc[i,'Stock Price'])

final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.10,40533280753,182
1,AAL,14.00,9009657868,1785
2,AAP,201.41,11814136920,124
3,AAPL,166.23,2689443502663,150
4,ABBV,150.22,262269902533,166
...,...,...,...,...
395,RL,101.64,7541509902,245
396,RMD,243.46,36301170259,102
397,ROK,264.21,29685602802,94
398,ROL,39.26,19041911938,636


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [31]:
writer=pd.ExcelWriter('recomended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [32]:
background_color = '#0a0a23'
font_color='#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border':1 #solid border around each cell
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0,00',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1 
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1 
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [33]:
#writer.sheets['Recommended Trades'].set_column('A:A', 18,string_format)
#writer.sheets['Recommended Trades'].set_column('B:B', 18,string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18,string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18,string_format)
#writer.save()


#writer.sheets['Recommended Trades'].write('A1','Ticker', string_format)
#writer.sheets['Recommended Trades'].write('B1','Stock Price', dollar__format)
#writer.sheets['Recommended Trades'].write('C1','Market Capitalization', dollar_format)
#writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify and replace the above by putting it in 2 loops:

In [34]:
column_formats = {
    'A': ['Ticker',string_format],
    'B': ['Stock Price',dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [35]:
writer.save()